In [1]:
from pulp import *
import numpy as np
import ArbitraryCode as ac

In [2]:
# Number of classes, players in various classes, Information Graph
k = 3;   #k should take vlaue in {2,3}
if k==2:
  c1 = 3;
  c2 = 4;
  C = np.array([c1,c2])
  N_mat = np.array([[1,  0],[1, 1]])
  n = np.sum(C)
  N = np.zeros((k,1))
elif k ==3:
  c1 = 2;
  c2 = 2;
  c3 = 2;
  C = np.array([c1,c2,c3])
  N_mat = np.array([[1, 0, 0],[1, 1,1],[1, 0,1]])
  n = np.sum(C)
  N = np.zeros((1,k))
else:
  raise Exception('choose k from {2,3}')

if np.trace(N_mat) !=k:
  raise Exception('Invalid information graph -- any class must be able to observe the agents in the same class')

for j in range(k):
    N[0,j] = np.sum(N_mat[j,:]*C)


In [3]:
# The basis function and utility design function
d = 0
w = np.arange(n+1)
w = np.power(w,d)

F = np.ones((k,n))
F[:,:] = np.nan

#last entry -- 1 for marginal contribution, 2 for equal share, 3 for all f being 1
for j in range(k):
  n = int(N[0,j])
  F[j,0:n] = ac.Generate_f(n,w,1)

In [17]:
def ArbitraryComputingPoA(C,N_mat,F,w):
  k = C.shape[0] 
  n = np.sum(C)
  N = np.zeros((1,k))
  for j in range(k):
    N[0,j] = np.sum(N_mat[j,:]*C)

  f0 = F[0,0:int(N[0,0])]
  f0 = np.append(0,f0)
  f0 = np.append(f0,0)
  f1 = F[1,0:int(N[0,1])]
  f1 = np.append(0,f1)
  f1 = np.append(f1,0)

  if k==3:
    f2 = F[2,0:int(N[0,2])]
    f2 = np.append(0,f2)
    f2 = np.append(f2,0)

  mm = int(np.max(N))
  f_lp = np.zeros((k,mm+2))

  f_lp[0,0:(int(N[0,0])+2)] = f0
  f_lp[1,0:(int(N[0,1])+2)] = f1
  f_lp[2,0:(int(N[0,2])+2)] = f2

  w_lp = np.append(0,w)

  #the objective function
  c = np.zeros((1,k))
  c = np.append(c,1)

  #the constraints
  IR = ac.ArbtitraryIR(C)
  numRowsIR = IR.shape[0]

  B= np.zeros((numRowsIR+k,1))
  A= np.zeros((numRowsIR+k,k+1))

  for j in range(k):
    A[-1-k+j,0] = -1
    B[-1-k+j]   = 0
  
  for t in range(554,numRowsIR):
    a = np.zeros((1,k))
    x = np.zeros((1,k))
    b = np.zeros((1,k))

    for j in range(k):
      idx = 3*(j+1)-3
      a[0,j] = IR[t,idx]
      x[0,j] = IR[t,idx+1]
      b[0,j] = IR[t,idx+2]
    
    At = np.sum(np.add(a,x))+1
    At = int(At)
    Bt = np.sum(np.add(b,x))+1
    Bt = int(Bt)

    Atj = np.zeros((1,k))
    for j in range(k):
      Atj[0,j] = np.sum(N_mat[j,:]*np.add(a,x))

    for j in range(k):
      temp_idx = int(Atj[0,j])
      print(t,temp_idx)
      A[t,j] = a[0,j]*f_lp[j,temp_idx+1] - b[0,j]*f_lp[j,temp_idx+2]  

    B[t] = - w_lp[Bt]
    A[t,k] = -w_lp[At] 
  
  return B

f = ArbitraryComputingPoA(C,N_mat,F,w)

print(f)

KeyboardInterrupt: 